# Computational Social Choice

This field explores and analyzes methods to aggregate collective choices made by numerous agents in a fair and efficient way. One such example is the 'plurality rule', used to define a president in the majority of the democratic countries.

## 1. Setting

The general setting of a computational social choice problem contains:
1. autonomous agents
2. a set of items (also called candidates or alternatives)
3. some preference of each agent towards each item
4. some voting rule that will be used to make a collective choice given the agents' preferences

_Example_:
- two agents: $a_1$ and $a_2$
- three items: A, B and C
- $a_1$ might have the following preference $A \geq B = C$
- $a_2$ might have the following preference $A \geq C \geq B$
- one example of a voting rule is the plurality rule, which means the group will choose the item that appears as a top-ranked choice most frequently (A in our case)

Key questions:
- how computationally expensive is it to compute collective choices?
- what is a rational choice? can we humans rank every set of items according to some underlying preference?
- given a voting rule, can votes manipulate their declared preferences in order to rig the voting process and avoid a terrible candidate?

## 2. Five Common Voting Rules

Before we explore some of the most common voting rules, let's assume that a population of 14 agents has the following preferene profile over items $a,b,c,d$:

In [41]:
CANDIDATES = set(['a', 'b', 'c', 'd', 'e'])

pref_dic = {
    5: ['a', 'c', 'b', 'd', 'e'], # 5 people rank a as highest, then c, ....
    4: ['e', 'b', 'c', 'd', 'a'], # 4 people rank e as highest, then b, ....
    3: ['d', 'c', 'b', 'e', 'a'], # 3 people rank d as highest, then c, ....
    2: ['b', 'd', 'e', 'c', 'a']  # 2 people rank b as highest, then d, ....
}

#### 2.1 Plurality

Pick the candidate that appears most frequently as the top-ranked choice across all agents. Note that this rule compeltely ignores the remaining preference relation (i.e., which candidate is second, third etc).

In [2]:
def plurality(pref_dic):
    candidates_prefs = {cand: 0 for cand in CANDIDATES}
    for nbr, prefs in pref_dic.items():
        candidates_prefs[prefs[0]] += nbr
    return max(candidates_prefs, key=candidates_prefs.get)

choice = plurality(pref_dic)
print(f"Plurality rule yields '{choice}' as the right choice for the group.")

Plurality rule yields 'a' as the right choice for the group.


#### 2.2 Borda's Rule

If there are $m$ candidates, ach agent gives a score $m - r$ to the candidate ranked in position $r$, and the candidate with the most votes is chosen.

In our example, $m=5$, so 'a' gets $5 \times (5 - 1) + 9 \times (5-5) = 20$ points.

In [3]:
def bordas(pref_dic):
    candidates_prefs = {cand: 0 for cand in CANDIDATES}
    for nbr, prefs in pref_dic.items():
        for i, cand in enumerate(prefs):
            candidates_prefs[cand] += nbr * (5-(i+1))
    print(f'scores: {candidates_prefs}')
    return max(candidates_prefs, key=candidates_prefs.get)

choice = bordas(pref_dic)
print(f"Borda's Rule yields '{choice}' as the right choice for the group.")

scores: {'e': 23, 'b': 36, 'd': 27, 'a': 20, 'c': 34}
Borda's Rule yields 'b' as the right choice for the group.


#### 2.3 Sequential Majority Comparison (SMC)

A plurality rule is applied in a pre-defined sequence of pair-wise comparison.

E.g., if the order is ((a vs b) vs c), then first the agents vote on a vs b, and then the winner gets to be compared with c to yield the final choice.

In [15]:
import numpy as np

def smc(pref_dic, order = ['a', 'b', 'c', 'd', 'e']):
    prev_winner = order[0]
    for i in range(1, len(order)):
        tally = [0,0]
        pair = [prev_winner, order[i]]
        for nbr, prefs in pref_dic.items():
            rank1 = prefs.index(pair[0])
            rank2 = prefs.index(pair[1])
            if rank1 < rank2:
                tally[0] += nbr
            else:
                tally[1] += nbr
        prev_winner = pair[np.argmax(tally)]
    return prev_winner

choice = smc(pref_dic)
print(f"SMC rule yields '{choice}' as the right choice for the group.")

SMC rule yields 'c' as the right choice for the group.


#### 2.4 - Instant-Runoff

The candidate with the least amount of top-ranked appearances is deleted, then the voting is done again. This happens until a single candidate is left standing.

In [78]:
from copy import deepcopy

def plurality_runoff(pref_dic):
    to_be_deleted = None
    pref_dic_cp = deepcopy(pref_dic)
    while len(pref_dic_cp[5])>1:
        candidates_prefs = {cand: 0 for cand in pref_dic_cp[5] if cand != to_be_deleted}
        for nbr, prefs in pref_dic_cp.items():
            if to_be_deleted:
                prefs.remove(to_be_deleted)
            candidates_prefs[prefs[0]] += nbr
        to_be_deleted = min(candidates_prefs, key=candidates_prefs.get)
    return max(candidates_prefs, key=candidates_prefs.get)
    
choice = plurality_runoff(pref_dic)
print(f"Instant-runoff rule yields '{choice}' as the right choice for the group.")

Instant-runoff rule yields 'd' as the right choice for the group.


#### 2.5 - Plurality with Runoff

First, a plurality rule is applied and the top two candidates face off in a second round.

In [27]:
def plurality_runoff(pref_dic):
    candidates_prefs = {cand: 0 for cand in CANDIDATES}
    for nbr, prefs in pref_dic.items():
        candidates_prefs[prefs[0]] += nbr
    top_2 = sorted(candidates_prefs, key=candidates_prefs.get, reverse=True)[:2]

    candidates_prefs = {cand: 0 for cand in top_2}
    for nbr, prefs in pref_dic.items():
        rank1 = prefs.index(top_2[0])
        rank2 = prefs.index(top_2[1])
        if rank1 < rank2:
            candidates_prefs[top_2[0]] += nbr
        else:
            candidates_prefs[top_2[1]] += nbr

    return max(candidates_prefs, key=candidates_prefs.get)
    
choice = plurality_runoff(pref_dic)
print(f"Plurality rule yields '{choice}' as the right choice for the group.")

Plurality rule yields 'e' as the right choice for the group.


## Challenges

As the previous exercise shows, the same preference profiles can lead to completely different outcomes depending on the voting rule. So, we must come up with a way to measure their fairness and effectiveness.


# Voting Axioms

In Social Choice Theory, we usually define what it means for a voting rule to be 'fair' by checking if it satisfies some common sense axioms. Two of them will be described below.

1. *Monotonicity*: If the picked choice is reinforced (i.e., some agents increase its ranking while keeping the other ones constant), then the new picked choice will be the same

2. *Pareto-Optimality*: If there is a candidate B that is preferred over candidate A across every single agent, then candidate A cannot be picked.